# This notebook plots the distribution of copper deposits in Papua New Guinea and associated ages. Data is from Holms et al., 2019

# https://www.sciencedirect.com/science/article/pii/S0169136817307783


<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0169136817307783-gr2_lrg.jpg" width="500" height="600">

In [1]:
import gplately

import pandas as pd
import numpy as np
from gplately import pygplates
import glob, os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

Presently, we still have to point to these data. In the future, a set of topologies, rotation files, age grids, etc. could be downloaded from a server.

In [2]:
gdownload = gplately.DataServer("Muller2019")
rotation_model, topology_features, static_polygons = gdownload.get_plate_reconstruction_files()
coastlines, continents, COBs = gdownload.get_topology_geometries()

Checking whether the requested files need to be updated...
Requested files are up-to-date!
Checking whether the requested files need to be updated...
Requested files are up-to-date!


## Making plate reconstructions

We simply supply a rotation model, plate topologies, and static polygons to initialise a plate reconstruction model.

In [3]:
model = gplately.PlateReconstruction(rotation_model, topology_features, static_polygons)
gplot = gplately.PlotTopologies(model, coastlines=coastlines, continents=continents, COBs=COBs)

## Uploading and reconstructing the data points
Now that we have defined our reconstruction object, we can reconstruct point data.

In [4]:
df = pd.read_csv('Holm_et_al_2019_Table_1_ed.csv', sep=',',header=0)

#Selecting the points by age group so that they appear only at a given times in the simulation
gpts1=gplately.Points(model, df['Longitude'][df.Age_group==1], df['Latitude'][df.Age_group==1])
gpts2=gplately.Points(model, df['Longitude'][df.Age_group==2], df['Latitude'][df.Age_group==2])
gpts3=gplately.Points(model, df['Longitude'][df.Age_group==3], df['Latitude'][df.Age_group==3])
gpts4=gplately.Points(model, df['Longitude'][df.Age_group==4], df['Latitude'][df.Age_group==4])
gpts5=gplately.Points(model, df['Longitude'][df.Age_group==5], df['Latitude'][df.Age_group==5])
gpts6=gplately.Points(model, df['Longitude'][df.Age_group==6], df['Latitude'][df.Age_group==6])
gpts7=gplately.Points(model, df['Longitude'][df.Age_group==7], df['Latitude'][df.Age_group==7])

#there are no deposits of ages between 16-20 Ma so we create two points that aren't in the extent of the map
gptse=gplately.Points(model, [0], [0])



## Plotting

The `PlotTopologies` function injests the plate model we have defined as well as the coastlines, continents, and COB. It computes all of the plate topologies for a given reconstruction time.

This object has been designed to work specifically with `cartopy`. Define your figure and supply your axes to these plotting routines. Some common favourites include:

- coastlines
- continents
- ridges and transforms
- trenches
- subduction teeth (!!)
- netCDF grids
- plate motion vectors

You can still supply optional keywords as you normally would.

In [5]:
#The following lines of code plot tectonic reconstructions every 1Myr and plot the location of ore deposits of a given age
#The age of the deposits are listed in the original dataset but the issue is that the ages are listed as an age range 
# e.g. df.Age_for_plot_Ma[df.Age_group==1]
#To relate the age of the deposit to the age of the reconstruction I use the lists ages and gpts
#Also the age ranges change in size and there are not copper deposits between 16-20Ma

ages=[0,0,0,0,1,1,1,2,2,2,3,3,3,4,4,4, 5,5,5,5,5,6,6,6,6,7,7,7]#[0-3, 3-6, 6-9, 9-12, 12-15, 16-20, 21-24, 24-27]Ma
gpts = [gpts1, gpts2, gpts3, gpts4, gpts5, gptse, gpts6, gpts7]

for time, i in enumerate(ages): 
    # Calculate velocity data at this time using flattened x and y mesh arrays
    Xnodes = np.arange(-180,180,5)
    Ynodes = np.arange(-90,90,5)

    # Generate the meshnode point feature and flatten back into 1d lat-lon arrays
    xnode_mesh,ynode_mesh = np.meshgrid(Xnodes,Ynodes)
    x_flattened = xnode_mesh.flatten()
    y_flattened = ynode_mesh.flatten()
    all_velocities = model.get_point_velocities(x_flattened, y_flattened, time, delta_time=1.0)
    
    # plot the map
    fig = plt.figure(figsize=(13,7.5),dpi=300)
    ax2 = fig.add_subplot(111, projection=ccrs.PlateCarree())
    ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                      xlocs=list(range(-180,180,10)), ylocs=list(range(-90,90,10)),
                      linewidth=1, color='gray', alpha=0.5, linestyle='-')
    ax2.set_extent([120, 170, 10, -22])
    # set this time on the gplot object
    gplot.time = time

    gplot.plot_continents(ax2, edgecolor='none', facecolor='0.90')
    gplot.plot_coastlines(ax2, edgecolor=(0,0,0,0.9), facecolor='none', linewidth=0.5)
    gplot.plot_trenches(ax2, linewidth=1.5)
    gplot.plot_subduction_teeth(ax2, spacing=0.01, size=0.5)
    gplot.plot_plate_motion_vectors(ax2, regrid_shape=10, alpha=0.2, color='green', zorder=2, 
                                    spacingX=5, spacingY=5)
    print(time, i)
    
    #Plot the paleo-location of the deposits
    rlons, rlats = gpts[i].reconstruct(time, return_array=True)
    ax2.scatter(rlons, rlats, transform=ccrs.PlateCarree(), zorder=10, color=(1,0,0,1)) 
    
    ax2.axes.set_title(str(time)+' Ma')
    plt.tight_layout()
    #Save each of the reconstructions in a separate file
    fig.savefig('PNG_plate_velocity_vectors%d_Ma.png' % time)
    plt.close(fig)

0 0
1 0
2 0
3 0
4 1
5 1
6 1
7 2
8 2
9 2
10 3
11 3
12 3
13 4
14 4
15 4
16 5
17 5
18 5
19 5
20 5
21 6
22 6
23 6
24 6
25 7
26 7
27 7


In [6]:
import glob

frame_list = []

#read all the files and use the timestamp and sort them in reverse order so that the videos are created in geologic order (old to young)
frame_list = sorted(glob.glob("PNG_plate_velocity_vectors*"), key=os.path.getmtime,  reverse=True)



In [7]:
import moviepy.editor as mpy

clip = mpy.ImageSequenceClip(frame_list, fps=2)
clip.write_gif('PNG_plate_velocity_vectors_red_notnom_1Myr_2fps.gif')



MoviePy - Building file PNG_plate_velocity_vectors_red_notnom_1Myr_2fps.gif with imageio.


# Plotting the velocity vectors as streamplot

In [8]:
# Extract the north and east velocity components and create u,v velocity components with them
pt_vel_n=[]
pt_vel_e=[]
for vel in all_velocities:
    pt_vel_e.append(vel[1])
    pt_vel_n.append(vel[0])
    
u = np.asarray(pt_vel_e).reshape((Ynodes.shape[0],Xnodes.shape[0]))
v = np.asarray(pt_vel_n).reshape((Ynodes.shape[0],Xnodes.shape[0]))
x = Xnodes
y = Ynodes

In [9]:
#Plotting the velocity vectors as streamplot
ages=[0,0,0,0,1,1,1,2,2,2,3,3,3,4,4,4, 5,5,5,5,5,6,6,6,6,7,7,7]#[0-3, 3-6, 6-9, 9-12, 12-15, 16-20, 21-24, 24-27]Ma
gpts = [gpts1, gpts2, gpts3, gpts4, gpts5, gptse, gpts6, gpts7]

gplot = gplately.plot.PlotTopologies(model, time, coastlines, continents, COBs)

for time, i in enumerate(ages): 
    Xnodes = np.arange(-180,180,5)
    Ynodes = np.arange(-90,90,5)

    # Generate the meshnode point feature and flatten back into 1d lat-lon arrays
    xnode_mesh,ynode_mesh = np.meshgrid(Xnodes,Ynodes)
    x_flattened = xnode_mesh.flatten()
    y_flattened = ynode_mesh.flatten()
    
    # Calculate velocity data at this time using flattened x and y mesh arrays
    all_velocities = model.get_point_velocities(x_flattened, y_flattened, time, delta_time=1.0)
    
    # Extract the north and east velocity components and recreate u,v velocity components with them at this time
    pt_vel_n=[]
    pt_vel_e=[]
    for vel in all_velocities:
        pt_vel_e.append(vel[1])
        pt_vel_n.append(vel[0])

    u = np.asarray(pt_vel_e).reshape((Ynodes.shape[0],Xnodes.shape[0]))
    v = np.asarray(pt_vel_n).reshape((Ynodes.shape[0],Xnodes.shape[0]))


    # plot the map
    fig = plt.figure(figsize=(16,12),dpi=300)
    ax2 = fig.add_subplot(111, projection=ccrs.PlateCarree())
    ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                      xlocs=list(range(-180,180,10)), ylocs=list(range(-90,90,10)),
                      linewidth=1, color='gray', alpha=0.5, linestyle='-')
    ax2.set_extent([120, 170, 10, -20])
    # set this time on the gplot object
    gplot.time = time

    gplot.plot_continents(ax2, edgecolor='none', facecolor='0.90')
    gplot.plot_coastlines(ax2, edgecolor=(0,0,0,0.9), facecolor='none', linewidth=0.5)
    gplot.plot_trenches(ax2, linewidth=1.5)
    gplot.plot_subduction_teeth(ax2, spacing=0.01, size=0.5)


    #Plot the paleo-location of the deposits
    rlons, rlats = gpts[i].reconstruct(time, return_array=True)
    ax2.scatter(rlons, rlats, transform=ccrs.PlateCarree(), zorder=10, color=(1,0,0,1)) 

    #Create the streamplot, using speed as a colormap.
    speed = np.sqrt(u**2 + v**2)
    S = ax2.streamplot(x,y,u,v,color=speed, transform=ccrs.PlateCarree(), 
                       linewidth=0.02*speed, cmap=plt.cm.rainbow, density=2)
    fig.colorbar(S.lines, shrink=0.5).set_label('Velocity magntitude (cm/yr)',fontsize=12)
    ax2.axes.set_title(str(time)+' Ma')
    fig.savefig('plate_velocity_stream_plot_%d_Ma.png' % time)
    plt.close(fig)

    print('Image for %d Ma saved' % time)


Image for 0 Ma saved
Image for 1 Ma saved
Image for 2 Ma saved
Image for 3 Ma saved
Image for 4 Ma saved
Image for 5 Ma saved
Image for 6 Ma saved
Image for 7 Ma saved
Image for 8 Ma saved
Image for 9 Ma saved
Image for 10 Ma saved
Image for 11 Ma saved
Image for 12 Ma saved
Image for 13 Ma saved
Image for 14 Ma saved
Image for 15 Ma saved
Image for 16 Ma saved
Image for 17 Ma saved
Image for 18 Ma saved
Image for 19 Ma saved
Image for 20 Ma saved
Image for 21 Ma saved
Image for 22 Ma saved
Image for 23 Ma saved
Image for 24 Ma saved
Image for 25 Ma saved
Image for 26 Ma saved
Image for 27 Ma saved
